In [1]:
import numpy as np
import pycbc.conversions as pc
import pycbc.pnutils as pnu
import pandas as pd
import h5py, os

from collections import defaultdict

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import random

import sys
from lal import YRJUL_SI

import seaborn as sns

import ast

In [2]:
def save_dict_to_hdf5(dic, filename):
    """
    ....
    """
    with h5py.File(filename, 'w') as h5file:
        recursively_save_dict_contents_to_group(h5file, '/', dic)

def recursively_save_dict_contents_to_group(h5file, path, dic):
    """
    ....
    """
    for key, item in dic.items():
        if isinstance(item, (np.ndarray, np.int64, np.float64, str, bytes)):
            h5file[path + key] = item
        elif isinstance(item, dict):
            recursively_save_dict_contents_to_group(h5file, path + key + '/', item)
        else:
            raise ValueError('Cannot save %s type'%type(item))

def load_dict_from_hdf5(filename):
    """
    ....
    """
    with h5py.File(filename, 'r') as h5file:
        return recursively_load_dict_contents_from_group(h5file, '/')

def recursively_load_dict_contents_from_group(h5file, path):
    """
    ....
    """
    ans = {}
    for key, item in h5file[path].items():
        if isinstance(item, h5py._hl.dataset.Dataset):
            ans[key] = item[()]
        elif isinstance(item, h5py._hl.group.Group):
            ans[key] = recursively_load_dict_contents_from_group(h5file, path + key + '/')
            
    return ans


def print_some_rcParams():
    rc_keys = ['backend', 'axes.labelsize', 'axes.titlesize', 'font.size',
        'legend.fontsize', 'xtick.labelsize', 'ytick.labelsize', 'font.family',
        'text.usetex', 'figure.figsize'] # 'font.sans-serif', 'font.serif'
    print('Using the following matplotlib settings:')
    for k in rc_keys:
        print(k, ':', plt.rcParams[k])

In [3]:
ifar_threshold = 10 ## yr 
number_injection = 10

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', None)

In [5]:
#This is the first file, all events are true
injection_file = '/home/bhooshan.gadre/work/O3/sub_solar_search/O3B_VTs/ssm-search/O3b_VTs/common_ssm_injections.hdf'
injections = load_dict_from_hdf5(injection_file)

In [6]:
df_true = pd.DataFrame(injections)

df_true = df_true.drop(columns=['coa_phase', 'declination', 'distance', 'gstlal_ifar', 'gstlal_weight', 'inclination', 'mbta_ifar', 'polarization', 
                                'right_ascension', 'snr_H', 'snr_L', 'snr_V', 'spin1x', 'spin1y', 'spin2x', 'spin2y', 'time'])

#df_true['label'] = 1

df_true = df_true.rename(columns={'mass2_source': 'mass2', 
                                  'mass1_source': 'mass1'})

df_true

,mass1,mass2,pycbc_ifar,spin1z,spin2z
0,1.692460,0.501600,0.0,-0.037187,0.007150
1,6.421466,0.763863,0.0,0.021298,0.038439
2,10.481894,1.045793,0.0,0.012784,-0.030501
3,8.906005,0.811978,0.0,-0.000504,-0.004068
4,7.129897,0.808812,0.0,-0.001127,-0.003302
...,...,...,...,...,...
1916578,5.784159,0.559250,0.0,-0.038265,-0.032295
1916579,8.621294,0.868851,0.0,0.032837,-0.010397
1916580,10.560706,0.717379,0.0,-0.029833,-0.031378
1916581,5.727628,0.886952,0.0,-0.024699,0.007844


In [7]:
background_file = '/home/lucashernan.quiceno/background/output.txt'

with open(background_file, 'r') as file:
    content = file.read()

# Convert the contents of the file into a Python dictionary
data_dict = ast.literal_eval(content.replace("defaultdict(<class 'list'>, ", "").rstrip(')'))

# Convert the dictionary to a DataFrame
df_false = pd.DataFrame(data_dict)

df_false

,mass1,mass2,spin1z,spin2z
0,0.504929,0.214580,-0.686235,-0.056092
1,1.489096,0.212058,-0.728668,0.024226
2,0.558495,0.204445,0.774588,0.006712
3,0.309922,0.200873,-0.099824,-0.078287
4,0.206215,0.206215,0.099892,0.099903
...,...,...,...,...
1916578,0.864843,0.247605,-0.416889,0.025385
1916579,0.501223,0.202115,-0.743296,-0.011618
1916580,0.956881,0.213006,-0.813010,0.049815
1916581,0.808391,0.312963,-0.449955,-0.030445


In [8]:
#this the background
trig_file = "/home/pycbc.offline/O3/production/O3A_SUB_SOLAR/sub_solar/STATMAP/H1L1V1-COMBINE_STATMAP_FULL_DATA-1238787802-853441.hdf"
bank_file = "/home/pycbc.offline/O3/production/O3A_SUB_SOLAR/sub_solar/BANK/H1L1V1-BANK2HDF-1238787802-853441.hdf"

In [9]:
bank = h5py.File(bank_file, 'r')
hashes = bank['template_hash'][:]

bank_tids = hashes.argsort()
fields = ['mass1', 'mass2', 'spin1z', 'spin2z']
trigs = h5py.File(trig_file, 'r')

trig_params = defaultdict(list)
for tid in trigs['background']['template_id'][:number_injection]:  ## test with 10 triggers. Later this can be [:] for all triggers
    bank_idx = np.argwhere(bank_tids == tid)[0]
    for k in fields:
        trig_params[k].append(bank[k][:][bank_idx][0])
#print(trig_params)

In [12]:
len(trigs['background']['template_id'][:])

39955275

In [13]:
len(trigs['background_h0']['H1L1']['ifar'][:])

16322600

In [10]:

df_false['pycbc_ifar'] = trigs['background_h0']['H1L1']['ifar'][:1916583]

df_false['label'] = 0
df_false

,mass1,mass2,spin1z,spin2z,pycbc_ifar,label
0,0.504929,0.214580,-0.686235,-0.056092,0.000314,0
1,1.489096,0.212058,-0.728668,0.024226,0.000070,0
2,0.558495,0.204445,0.774588,0.006712,0.000018,0
3,0.309922,0.200873,-0.099824,-0.078287,0.000821,0
4,0.206215,0.206215,0.099892,0.099903,0.000288,0
...,...,...,...,...,...,...
1916578,0.864843,0.247605,-0.416889,0.025385,0.000010,0
1916579,0.501223,0.202115,-0.743296,-0.011618,0.000011,0
1916580,0.956881,0.213006,-0.813010,0.049815,0.000470,0
1916581,0.808391,0.312963,-0.449955,-0.030445,0.000045,0


In [11]:
df = pd.concat([df_true, df_false], axis=0)
df

,mass1,mass2,pycbc_ifar,spin1z,spin2z,label
0,1.692460,0.501600,0.000000,-0.037187,0.007150,NaN
1,6.421466,0.763863,0.000000,0.021298,0.038439,NaN
2,10.481894,1.045793,0.000000,0.012784,-0.030501,NaN
3,8.906005,0.811978,0.000000,-0.000504,-0.004068,NaN
4,7.129897,0.808812,0.000000,-0.001127,-0.003302,NaN
...,...,...,...,...,...,...
1916578,0.864843,0.247605,0.000010,-0.416889,0.025385,0.0
1916579,0.501223,0.202115,0.000011,-0.743296,-0.011618,0.0
1916580,0.956881,0.213006,0.000470,-0.813010,0.049815,0.0
1916581,0.808391,0.312963,0.000045,-0.449955,-0.030445,0.0


# Analisys with true data

In [12]:
df_true.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1916583 entries, 0 to 1916582
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   mass1       float64
 1   mass2       float64
 2   pycbc_ifar  float64
 3   spin1z      float64
 4   spin2z      float64
dtypes: float64(5)
memory usage: 73.1 MB


In [13]:
df_true.describe()

,mass1,mass2,pycbc_ifar,spin1z,spin2z
count,1.916583e+06,1.916583e+06,1.916583e+06,1.916583e+06,1.916583e+06
mean,5.630262e+00,6.863977e-01,6.351753e+02,2.688936e-04,-1.911760e-06
std,2.906692e+00,2.533340e-01,5.483003e+03,3.329348e-02,3.330472e-02
min,1.958331e-01,1.900007e-01,0.000000e+00,-9.981075e-02,-9.991923e-02
25%,3.132707e+00,4.788347e-01,0.000000e+00,-1.827353e-02,-1.861429e-02
50%,5.491139e+00,7.022830e-01,0.000000e+00,8.221344e-05,7.060659e-06
75%,8.075941e+00,9.049654e-01,0.000000e+00,1.894186e-02,1.864963e-02
max,1.099997e+01,1.100000e+00,1.427988e+05,9.988072e-02,9.988070e-02


In [14]:
columns_equal_0 = (df_true['pycbc_ifar'] == 0).sum()
print(f"Number of columns in 'ifar_true' equal than 0: {columns_equal_0}")

Number of columns in 'ifar_true' equal than 0: 1845407


In [15]:
print(f"Percentage of 'ifar_true' equal to 0: {columns_equal_0/len(df_true)}")

Percentage of 'ifar_true' equal to 0: 0.9628630745446454


In [16]:
columns_greater_than_10 = (df_true['pycbc_ifar'] > 10).sum()
print(f"Number of columns in 'ifar_true' greater than 10: {columns_greater_than_10}")

Number of columns in 'ifar_true' greater than 10: 63312


In [17]:
print(f"Percentage of 'ifar_true' greater than 10: {columns_greater_than_10/len(df_true)}")

Percentage of 'ifar_true' greater than 10: 0.03303378982282531


In [18]:
data_true = np.load('resultados_modelo_true_data.npz')

# Acceder a cada array almacenado
y_train = data_true['y_train']
y_pred = data_true['y_pred']
y_test = data_true['y_test']
mape_values = data_true['mape_values']
predictions = data_true['predictions']

In [19]:
(y_train > 10).sum()

50605

In [20]:
len(y_train), len(y_test), len(y_pred)

(1533266, 383317, 383317)

In [21]:
pd.Series(y_test).describe()

count    383317.000000
mean        628.643870
std        5477.191202
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max      142798.765113
dtype: float64

In [22]:
columns_equal_0_test = (y_test == 0).sum()
print(f"Number of columns in 'y_test' equal  0: {columns_equal_0_test}")

Number of columns in 'y_test' equal  0: 369054


In [23]:
print(f"Percentage of 'y_test' equal to 0: {columns_equal_0_test/len(y_test)}")

Percentage of 'y_test' equal to 0: 0.9627905884685521


In [24]:
columns_greater_than_10_test = (y_test > 10).sum()
print(f"Number of columns in 'y_test' greater than 10: {columns_greater_than_10_test}")

Number of columns in 'y_test' greater than 10: 12707


In [25]:
print(f"Percentage of 'y_test' greater than 10: {columns_greater_than_10_test/len(y_test)}")

Percentage of 'y_test' greater than 10: 0.033150108135042274


In [26]:
pd.Series(y_pred).describe()

count    383317.000000
mean        706.805888
std        1081.515994
min           0.000000
25%           0.207592
50%         253.240645
75%        1024.639935
max       41674.510798
dtype: float64

In [27]:
columns_equal_0_pred = (y_pred == 0).sum()
print(f"Number of columns in 'y_pred' equal  0: {columns_equal_0_pred}")

Number of columns in 'y_pred' equal  0: 81814


In [28]:
print(f"Percentage of 'y_pred' equal to 0: {columns_equal_0_pred/len(y_pred)}")

Percentage of 'y_pred' equal to 0: 0.2134369203557369


In [29]:
columns_greater_than_10_pred = (y_pred > 10).sum()
print(f"Number of columns in 'y_test' greater than 10: {columns_greater_than_10_pred}")

Number of columns in 'y_test' greater than 10: 268856


In [30]:
print(f"Percentage of 'y_test' greater than 10: {columns_greater_than_10_pred/len(y_pred)}")

Percentage of 'y_test' greater than 10: 0.7013933637172366


# New Analysis with ifar > 10

In [54]:
df_true_10 = df_true[df_true['pycbc_ifar'] > 10]
df_true_10

,mass1,mass2,pycbc_ifar,spin1z,spin2z
17,10.044697,0.771064,10174.474264,0.003799,-0.034952
39,9.871082,0.608848,10174.474264,0.003390,0.002160
45,6.012620,0.290126,10174.474264,0.022516,0.025649
52,1.566219,0.522978,10174.474264,0.032162,-0.004396
59,5.331335,0.458099,10174.474264,-0.004934,0.009750
...,...,...,...,...,...
1903266,6.626040,0.982731,69.997932,-0.030900,-0.025199
1903269,10.661847,0.462810,25146.646963,0.011288,0.066585
1903305,2.810979,0.813847,54.111754,0.012777,-0.049325
1903359,8.898606,0.641294,25146.646963,-0.067872,-0.015317


In [55]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import seaborn as sns

In [56]:
df_true_10.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63312 entries, 17 to 1903364
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   mass1       63312 non-null  float64
 1   mass2       63312 non-null  float64
 2   pycbc_ifar  63312 non-null  float64
 3   spin1z      63312 non-null  float64
 4   spin2z      63312 non-null  float64
dtypes: float64(5)
memory usage: 2.9 MB


In [57]:
df_true_10.describe()

,mass1,mass2,pycbc_ifar,spin1z,spin2z
count,63312.000000,63312.000000,63312.000000,63312.000000,63312.000000
mean,5.383882,0.684438,19227.671304,0.000500,-0.000093
std,2.847447,0.247258,23507.359703,0.033351,0.033146
min,0.226180,0.190010,10.005148,-0.099592,-0.099779
25%,2.950139,0.484471,7892.849440,-0.018034,-0.018864
50%,5.208983,0.700131,10776.944676,0.000157,-0.000059
75%,7.699286,0.893496,17538.416094,0.019170,0.018613
max,10.999429,1.099973,142798.765113,0.098532,0.099760


In [58]:
def evaluate_random_forest(X, y, n_estimators_values):
    # Dividir los datos en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Definir y entrenar el modelo
    model = RandomForestRegressor(n_estimators=n_estimators_values, random_state=42)
    model.fit(X_train, y_train)
    
    # Hacer predicciones sobre el conjunto de prueba
    y_pred = model.predict(X_test)
    
    # Calcular métricas
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Calcular el porcentaje de predicciones superiores a un umbral
    ifar_threshold = 10  # Se asume que este es el umbral que deseas utilizar
    prediction_value = (y_pred > ifar_threshold).sum() / len(y_test) * 100
    
    # Imprimir los resultados
    print(f"len(y_test) con n_estimators_values={n_estimators_values}: {len(y_test)}")
    print(f"(y_test > 10).sum() con n_estimators_values={n_estimators_values}: {(y_test > ifar_threshold).sum()}")
    print(f"(y_pred > 10).sum() con n_estimators_values={n_estimators_values}: {(y_pred > ifar_threshold).sum()}")
    print(f"predicción con n_estimators_values={n_estimators_values}: {prediction_value:.2f}%")
    print(f"MSE con n_estimators_values={n_estimators_values}: {mse:.4f}")
    print(f"RMSE con n_estimators_values={n_estimators_values}: {rmse:.4f}")
    print(f"MAE con n_estimators_values={n_estimators_values}: {mae:.4f}")
    print(f"MAPE con n_estimators_values={n_estimators_values}: {mape:.4f}")
    print(f"R^2 con n_estimators_values={n_estimators_values}: {r2:.4f}")
    print("-" * 40)

# n_estimators_values = 3

In [59]:
X = df_true_10[['mass1', 'mass2', 'spin1z', 'spin2z']]
y = df_true_10['pycbc_ifar']
n_estimators_values = 3

evaluate_random_forest(X, y, n_estimators_values)

len(y_test) con n_estimators_values=3: 12663
(y_test > 10).sum() con n_estimators_values=3: 12663
(y_pred > 10).sum() con n_estimators_values=3: 12663
predicción con n_estimators_values=3: 100.00%
MSE con n_estimators_values=3: 788522819.8761
RMSE con n_estimators_values=3: 28080.6485
MAE con n_estimators_values=3: 20165.4831
MAPE con n_estimators_values=3: 74.3200
R^2 con n_estimators_values=3: -0.4246
----------------------------------------


# n_estimators_values = 20

In [60]:
X = df_true_10[['mass1', 'mass2', 'spin1z', 'spin2z']]
y = df_true_10['pycbc_ifar']
n_estimators_values = 20

evaluate_random_forest(X, y, n_estimators_values)

len(y_test) con n_estimators_values=20: 12663
(y_test > 10).sum() con n_estimators_values=20: 12663
(y_pred > 10).sum() con n_estimators_values=20: 12663
predicción con n_estimators_values=20: 100.00%
MSE con n_estimators_values=20: 608373197.8849
RMSE con n_estimators_values=20: 24665.2224
MAE con n_estimators_values=20: 18226.8428
MAPE con n_estimators_values=20: 73.4676
R^2 con n_estimators_values=20: -0.0991
----------------------------------------


# n_estimators_values = 100

In [61]:
X = df_true_10[['mass1', 'mass2', 'spin1z', 'spin2z']]
y = df_true_10['pycbc_ifar']
n_estimators_values = 100

evaluate_random_forest(X, y, n_estimators_values)

len(y_test) con n_estimators_values=100: 12663
(y_test > 10).sum() con n_estimators_values=100: 12663
(y_pred > 10).sum() con n_estimators_values=100: 12663
predicción con n_estimators_values=100: 100.00%
MSE con n_estimators_values=100: 582439791.0876
RMSE con n_estimators_values=100: 24133.7894
MAE con n_estimators_values=100: 18008.7550
MAPE con n_estimators_values=100: 73.5853
R^2 con n_estimators_values=100: -0.0523
----------------------------------------


# Other dataframe. Original dataframe with all data df, true and false and then make the cut 

In [62]:
df_all_10 = df[df['pycbc_ifar'] > 10]
df_all_10

,mass1,mass2,pycbc_ifar,spin1z,spin2z,label
17,10.044697,0.771064,10174.474264,0.003799,-0.034952,NaN
39,9.871082,0.608848,10174.474264,0.003390,0.002160,NaN
45,6.012620,0.290126,10174.474264,0.022516,0.025649,NaN
52,1.566219,0.522978,10174.474264,0.032162,-0.004396,NaN
59,5.331335,0.458099,10174.474264,-0.004934,0.009750,NaN
...,...,...,...,...,...,...
1894031,1.393201,0.215449,24.438800,-0.809985,0.041113,0.0
1900545,0.724096,0.245961,14.287633,0.733550,-0.012316,0.0
1904515,0.211373,0.211373,111.671741,0.096349,0.013501,0.0
1906231,5.292538,0.577691,31.285468,-0.679952,-0.791074,0.0


In [63]:
X = df_all_10[['mass1', 'mass2', 'spin1z', 'spin2z']]
y = df_all_10['pycbc_ifar']
n_estimators_values = 10

evaluate_random_forest(X, y, n_estimators_values)

len(y_test) con n_estimators_values=10: 12734
(y_test > 10).sum() con n_estimators_values=10: 12734
(y_pred > 10).sum() con n_estimators_values=10: 12734
predicción con n_estimators_values=10: 100.00%
MSE con n_estimators_values=10: 642600673.8685
RMSE con n_estimators_values=10: 25349.5695
MAE con n_estimators_values=10: 18529.7483
MAPE con n_estimators_values=10: 72.9841
R^2 con n_estimators_values=10: -0.1527
----------------------------------------


In [64]:
X = df_all_10[['mass1', 'mass2', 'spin1z', 'spin2z']]
y = df_all_10['pycbc_ifar']
n_estimators_values = 100

evaluate_random_forest(X, y, n_estimators_values)

len(y_test) con n_estimators_values=100: 12734
(y_test > 10).sum() con n_estimators_values=100: 12734
(y_pred > 10).sum() con n_estimators_values=100: 12734
predicción con n_estimators_values=100: 100.00%
MSE con n_estimators_values=100: 587032930.7422
RMSE con n_estimators_values=100: 24228.7625
MAE con n_estimators_values=100: 18027.0655
MAPE con n_estimators_values=100: 74.2746
R^2 con n_estimators_values=100: -0.0530
----------------------------------------


# Other datafram. true>10 and false>10 and then concave

In [68]:
df_false_10 = df_false[df_false['pycbc_ifar'] > 10]
df_false_10

,mass1,mass2,spin1z,spin2z,pycbc_ifar,label
6239,1.129211,0.494514,-0.899860,-0.072271,10.197039,0
7751,0.547399,0.217088,0.441201,-0.037319,10.022269,0
8683,0.278654,0.224332,0.064740,0.078906,16.918181,0
11147,0.399051,0.200079,-0.099922,-0.099576,183.779780,0
14208,0.357954,0.201382,-0.097487,-0.097118,11.724922,0
...,...,...,...,...,...,...
1894031,1.393201,0.215449,-0.809985,0.041113,24.438800,0
1900545,0.724096,0.245961,0.733550,-0.012316,14.287633,0
1904515,0.211373,0.211373,0.096349,0.013501,111.671741,0
1906231,5.292538,0.577691,-0.679952,-0.791074,31.285468,0


In [69]:
df__cut = pd.concat([df_true_10, df_false_10], axis=0)
df__cut

,mass1,mass2,pycbc_ifar,spin1z,spin2z,label
17,10.044697,0.771064,10174.474264,0.003799,-0.034952,NaN
39,9.871082,0.608848,10174.474264,0.003390,0.002160,NaN
45,6.012620,0.290126,10174.474264,0.022516,0.025649,NaN
52,1.566219,0.522978,10174.474264,0.032162,-0.004396,NaN
59,5.331335,0.458099,10174.474264,-0.004934,0.009750,NaN
...,...,...,...,...,...,...
1894031,1.393201,0.215449,24.438800,-0.809985,0.041113,0.0
1900545,0.724096,0.245961,14.287633,0.733550,-0.012316,0.0
1904515,0.211373,0.211373,111.671741,0.096349,0.013501,0.0
1906231,5.292538,0.577691,31.285468,-0.679952,-0.791074,0.0


In [70]:
X = df__cut[['mass1', 'mass2', 'spin1z', 'spin2z']]
y = df__cut['pycbc_ifar']
n_estimators_values = 100

evaluate_random_forest(X, y, n_estimators_values)

len(y_test) con n_estimators_values=100: 12734
(y_test > 10).sum() con n_estimators_values=100: 12734
(y_pred > 10).sum() con n_estimators_values=100: 12734
predicción con n_estimators_values=100: 100.00%
MSE con n_estimators_values=100: 587032930.7422
RMSE con n_estimators_values=100: 24228.7625
MAE con n_estimators_values=100: 18027.0655
MAPE con n_estimators_values=100: 74.2746
R^2 con n_estimators_values=100: -0.0530
----------------------------------------
